In [1]:
# -*- coding: utf-8 -*-

from __future__ import print_function, division
from util import *
import os
import sys
import subprocess
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from imutils import face_utils
import imutils
import dlib
import cv2
import json
from shutil import rmtree
from config import affectnet_config as config
from pyimagesearch.io import DatasetGenerator
from pyimagesearch.preprocessing import AspectAwarePreprocessor, ImageToArrayPreprocessor, SimplePreprocessor, MeanPreprocessor, CropPreprocessor
from keras.models import load_model

Using TensorFlow backend.
/home/a/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/anaconda3/lib/python3.7/site-packages/tensorflow/python/framewo

In [3]:

images = glob('/path/to/images/jpg/A/*/*.jpg')
print('Found {} images'.format( len(images)))

Found 81644 images


In [4]:
means = json.loads(open(config.DATASET_MEAN).read())
sp = SimplePreprocessor(227, 227)
mp = MeanPreprocessor(means['R'], means['G'], means['B'])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

#load the pretrained network
model = load_model(config.MODEL_PATH)

In [5]:
predictions = []
for image in tqdm(images[:100]):
    face = face_get(image)
    crop = sp.preprocess(face)
    crop = mp.preprocess(crop)
    crop = np.expand_dims(crop, axis=0)
    #crop = np.array([iap.preprocess(c) for c in crop], dtype = "float32")
    pred = model.predict(crop)
    predictions.append([image, pred.mean(axis=0)])


100%|██████████| 100/100 [00:03<00:00, 25.04it/s]


In [6]:
neu = [pred[1][0] for pred in predictions]
hap = [pred[1][1] for pred in predictions]
sad = [pred[1][2] for pred in predictions]
ang = [pred[1][3] for pred in predictions]
imgs = [pred[0] for pred in predictions]

In [7]:
df = pd.DataFrame({'image':imgs, 'Neutral':neu, 'Happy':hap, 'Sad':sad, 'Angry':ang})
df = df.sort_values(['image'])
df.to_csv("results.csv", index=False)